In [18]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
from pandas.api.indexers import FixedForwardWindowIndexer

In [19]:
#========
# Setup
#========
np.random.seed(928)

dates = pd.date_range("2025-12-01" , periods = 14 , freq = "D")

df = pd.DataFrame({
    "date": np.tile(dates , 2) ,
    "region": ["East"] * 14 + ["West"] * 14 ,
})

base = np.linspace(100 , 130 , 14)
df["sales"] = np.r_[base + np.random.normal(0 , 6 , 14) ,
                   base + 8 + np.random.normal(0 , 7 , 14)].round(0).astype(int)
df["returns"] = np.r_[np.random.poisson(4 , 14) , np.random.poisson(5 , 14)]
df.head()

ts = df.sort_values(["region", "date"]).set_index("date")
ts.head()

,date,region,sales,returns
0,2025-12-01,East,110,4
1,2025-12-02,East,107,5
2,2025-12-03,East,104,7
3,2025-12-04,East,105,3
4,2025-12-05,East,106,3


,region,sales,returns
date,,,
2025-12-01,East,110,4
2025-12-02,East,107,5
2025-12-03,East,104,7
2025-12-04,East,105,3
2025-12-05,East,106,3


In [20]:
#==================================================
# Case 1) EWM basics: exponentially weighted mean
#==================================================
ts["sales_ewm_span4"] = (
    ts.groupby("region")["sales"]
        .transform(lambda s: s.ewm(span = 4 , adjust = False).mean())
)
ts[["region" , "sales" , "sales_ewm_span4"]].head(10).round(2)

,region,sales,sales_ewm_span4
date,,,
2025-12-01,East,110,110.00
2025-12-02,East,107,108.80
2025-12-03,East,104,106.88
2025-12-04,East,105,106.13
2025-12-05,East,106,106.08
2025-12-06,East,116,110.05
2025-12-07,East,116,112.43
2025-12-08,East,112,112.26
2025-12-09,East,122,116.15


In [21]:
#============================================================
# Case 2) Decay control: compare span vs alpha
# alpha close to 1 => reacts fast; alpha small => smoother
#============================================================
ts["sales_ewm_alpha_06"] = (
    ts.groupby("region")["sales"]
        .transform(lambda s: s.ewm(alpha = 0.6 , adjust = False).mean())
)
ts[["region" , "sales" , "sales_ewm_span4" , "sales_ewm_alpha_06"]].head(10).round(2)

,region,sales,sales_ewm_span4,sales_ewm_alpha_06
date,,,,
2025-12-01,East,110,110.00,110.00
2025-12-02,East,107,108.80,108.20
2025-12-03,East,104,106.88,105.68
2025-12-04,East,105,106.13,105.27
2025-12-05,East,106,106.08,105.71
2025-12-06,East,116,110.05,111.88
2025-12-07,East,116,112.43,114.35
2025-12-08,East,112,112.26,112.94
2025-12-09,East,122,116.15,118.38


In [22]:
#=================================================
# Case 3) Irregular timestamps: times + halflife
#=================================================
ir = ts.reset_index().copy()
ir = ir[~ir["date"].dt.day.isin([3 , 7 , 11])]
ir = ir.sort_values(["region" , "date"]).set_index("date")

ir["sales_ewm_hl_4d"] = (
    ir.groupby("region")["sales"]
        .transform(lambda s: s.ewm(halflife = "4 days" , times = s.index , adjust = True)
                   .mean())
)

ir[["region" , "sales" , "sales_ewm_hl_4d"]].head(10).round(2)

,region,sales,sales_ewm_hl_4d
date,,,
2025-12-01,East,110,110.00
2025-12-02,East,107,108.37
2025-12-04,East,105,106.91
2025-12-05,East,106,106.60
2025-12-06,East,116,109.31
2025-12-08,East,112,110.09
2025-12-09,East,122,113.14
2025-12-10,East,121,114.98
2025-12-12,East,143,121.93


In [23]:
#======================================================
# Case 4) EWM volatility: exponentially weighted std
#======================================================
ts["returns_ewm_std_span5"] = (
    ts.groupby("region")["returns"]
        .transform(lambda s: s.ewm(span = 5 , adjust = False).std(bias = False))
)
ts[["region" , "returns" , "returns_ewm_std_span5"]].head(12).round(2)

,region,returns,returns_ewm_std_span5
date,,,
2025-12-01,East,4,NaN
2025-12-02,East,5,0.71
2025-12-03,East,7,1.64
2025-12-04,East,3,1.76
2025-12-05,East,3,1.61
2025-12-06,East,4,1.30
2025-12-07,East,9,2.85
2025-12-08,East,2,3.02
2025-12-09,East,3,2.58


In [24]:
#============================================================
# Case 5) Weighted windows (finite window, custom weights)
#============================================================
weights = np.arange(1 , 6)

def wma_last5(x: np.ndarray) -> float:
    w = weights[-len(x):]
    return float(np.dot(x , w) / w.sum())

ts["sales_wma_5"] = (
    ts.groupby("region")["sales"].rolling(window = 5 , min_periods = 1)
        .apply(wma_last5 , raw = True).reset_index(level = 0 , drop = True)
)
ts[["region" , "sales" , "sales_wma_5"]].head(10).round(2)

,region,sales,sales_wma_5
date,,,
2025-12-01,East,110,110.00
2025-12-02,East,107,108.33
2025-12-03,East,104,106.50
2025-12-04,East,105,105.86
2025-12-05,East,106,105.73
2025-12-06,East,116,108.93
2025-12-07,East,116,111.73
2025-12-08,East,112,112.60
2025-12-09,East,122,116.27


In [25]:
#=======================================================
# Case 6) Intermediate trick: Forward-looking windows
# Next-3-days average sales (useful for labels/targets)
#=======================================================
fwd3 = FixedForwardWindowIndexer(window_size = 3)
tmp = ts.reset_index().sort_values(["region" , "date"]).set_index("date")
tmp["sales_next3_mean"] = (
    tmp.groupby("region")["sales"].rolling(window = fwd3 , min_periods = 1)
        .mean().reset_index(level = 0 , drop = True)
)

ts["sales_next3_mean"] = tmp["sales_next3_mean"].values
ts[["region" , "sales" , "sales_next3_mean"]].head(12).round(2)

,region,sales,sales_next3_mean
date,,,
2025-12-01,East,110,107.00
2025-12-02,East,107,105.33
2025-12-03,East,104,105.00
2025-12-04,East,105,109.00
2025-12-05,East,106,112.67
2025-12-06,East,116,114.67
2025-12-07,East,116,116.67
2025-12-08,East,112,118.33
2025-12-09,East,122,121.00
